In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import os
import sys

project_path = os.path.abspath(os.path.join('..'))

if project_path not in sys.path:
    sys.path.append(project_path)

In [20]:
from src.features.generate_features import *
from src.models.model_pipeline import *
from src.preprocess.preprocess_ATSF import process
## change path
# from model_pipeline import *

In [21]:
data = pd.read_csv('../data/kaggle/data_kaggle.csv')
data = process(data)

In [22]:
config_filename = '../src/configs/gen_features_config.json'
model_config_filename = '../src/configs/model_config.json'

In [23]:
features_json = read_json(config_filename)
data[features_json['date_col']] = pd.to_datetime(data[features_json['date_col']], dayfirst=True)

In [24]:
data_features = generate_lagged_features(data=data,
                         target_cols=features_json['target_cols'],
                         id_cols=features_json['id_cols'],
                         date_col=features_json['date_col'],
                         lags=features_json['lags'],
                         windows=features_json['windows'],
                         preagg_methods=features_json['preagg_methods'],
                         agg_methods=features_json['agg_methods'],
                         dynamic_filters=features_json['dynamic_filters'],
                         ewm_params=features_json['ewm_params']
                        )

IntProgress(value=0, max=1)

In [25]:
choose_model(model_config_filename)

Choose model:


interactive(children=(Dropdown(description='model', options=('LSTM', 'GRU', 'TFT', 'DeepAR', 'NBeats'), value=…

In [27]:
data_features

,Date,Store_id,SKU_id,Promo,Demand,Regular_Price,Actual_Price,Promo_percent,weekday,weekday_name,monthday,is_weekend,month_period,demand_expanding_mean,Demand_lag14d_w14D_keySKU_id_Store_id_preagmean_agmean_Promo_dynamic_rolling
0,2015-01-01,1,1,0.0,22.0,163.78,163.78,0.000000,3,Thursday,1,0,0,NaN,NaN
1,2015-01-01,53,1,0.0,3.0,163.78,163.78,0.000000,3,Thursday,1,0,0,NaN,NaN
2,2015-01-01,54,1,0.0,6.0,163.78,163.78,0.000000,3,Thursday,1,0,0,NaN,NaN
3,2015-01-01,49,1,0.0,6.0,163.78,163.78,0.000000,3,Thursday,1,0,0,NaN,NaN
4,2015-01-01,55,1,0.0,16.0,163.78,163.78,0.000000,3,Thursday,1,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92881,2016-06-19,92,2,0.0,NaN,133.06,133.06,0.000000,6,Sunday,19,1,1,25.199029,NaN
92882,2016-06-19,66,1,1.0,NaN,134.96,125.04,0.073503,6,Sunday,19,1,1,256.120079,NaN
92883,2016-06-19,30,1,1.0,NaN,134.96,125.04,0.073503,6,Sunday,19,1,1,211.535433,NaN
92884,2016-06-19,76,1,1.0,NaN,134.96,125.04,0.073503,6,Sunday,19,1,1,183.439834,NaN


In [26]:
model_config = read_json(model_config_filename)
res_test = pipeline(data_features, model_config)

Preprocessing:


  0%|          | 0/2860 [00:00<?, ?it/s]

TypeError: Could not convert ThursdayFridaySaturdaySundayMondayTuesdayWednesdayThursdayFridaySaturdaySundayMondayTuesdayWednesdayThursdayFridaySaturdaySundayMondayTuesdayWednesdayThursdayFridaySaturdaySundayMondayTuesdayWednesdayThursdayFridaySaturdaySundayMondayTuesdayWednesdayThursdayFridaySaturdaySundayMondayTuesdayWednesdayThursdayFridaySaturdaySundayMondayTuesdayWednesdayThursdayFridaySaturdaySundayMondayTuesdayWednesdayThursdayFridaySaturdaySundayMondayTuesdayWednesdayThursdayFridaySaturdaySundayMondayTuesdayWednesdayThursdayFridaySaturdaySundayMondayTuesdayWednesdayThursdayFridaySaturdaySundayMondayTuesdayWednesdayThursdayFridaySaturdaySundayMondayTuesdayWednesdayThursdayFridaySaturdaySundayMondayTuesdayWednesdayThursdayFridaySaturdaySundayMondayTuesdayWednesdayThursdayFridaySaturdaySundayMondayTuesdayWednesdayThursdayFridaySaturdaySundayMondayTuesdayWednesdayThursdayFridaySaturdaySundayMondayTuesdayWednesdayThursdayFridaySaturdaySundayMondayTuesdayWednesdayThursdayFridaySaturda

In [9]:
res_test

,SKU_id,Store_id,Date,Demand
0,1,1,2016-05-23,156.448517
1,1,1,2016-05-24,45.283524
2,1,1,2016-05-25,52.282661
3,1,1,2016-05-26,50.207172
4,1,1,2016-05-27,52.015919
...,...,...,...,...
5965,2,110,2016-06-15,11.916748
5966,2,110,2016-06-16,7.261536
5967,2,110,2016-06-17,43.00148
5968,2,110,2016-06-18,19.580734


In [10]:
res_test[model_config['target']].to_csv('ans.csv', index_label='id')